# Solve the 1D Harmonic Oscillator problem as a PDE boundary value problem
In this example, we use atomic units. Mass unit is electron mass $m_e$=1. $\hbar=1$. Energy unit is Hartree $E_h$=1. Length unit is bohr.

Particle mass $m$ (in $m_e$), angular frequqnecy $\omega$ (unitless) will be input parameters of the NumerovSolverHO class.

$$\psi''=-\frac{2m}{\hbar^2}[E-V(x)]\psi$$

$$\psi(-\infty)=0$$

$$\psi(\infty)=0$$

where $$V(x) = \frac{1}{2}m\omega^2x^2$$

We are going to solve it numerically as a PDE boundary value problem. The algorithm is given by 

Donald Truhlar, Journal of Computational Physics 10, (1972)


For simplicity, we assume that $\mu=0.5$, $\omega=2$. Let npoints be the number of grid points, $h$=range/npoints.

To write the problem in the form of the paper, we have
$$U_{ii}^h =  -2h^2 V(x)$$
$$\lambda_k^h = -2h^2\epsilon_k$$


In [153]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt 
import time

hbar = 1
class PDE_BV:
    def __init__(self,m,omega, L, npoints=1000):
        self.m = m
        self.omega = omega
        self.L = L
        self.xlower = -L*0.5
        self.xupper = L*0.5
        self.npoints = npoints
        self.x = np.linspace(self.xlower,self.xupper,self.npoints)
        self.h = self.x[1]-self.x[0]
        self.psi = [None]*self.npoints
        self.E = np.zeros(self.npoints)
    
    def getV(self, x):
        return 0.5*self.m*self.omega**2*x**2
        
    def getUii(self, i):
        return  -2*(self.h**2)*self.getV(self.x[i])
    
    def getF(self):
        F = np.zeros([self.npoints,self.npoints])
        for i in range(0,self.npoints):
            F[i,i] = self.getUii(i) - 2/self.m
            if i > 0:
                F[i,i-1] = 1/self.m
            if i < self.npoints-1:
                F[i,i+1] = 1/self.m
                
        return F
 
    def diagonalize(self):
        F = self.getF()
        w,v = np.linalg.eig(F)
        return w,v
 
    
    def calc(self):
        w,v = self.diagonalize()
        self.E = - w/2.0/self.h**2
        #print(w)
        #print(self.E[0])
        for k in range(0,len(w)):
            self.psi[k] = v[:,k]
            integral = self.h*np.dot(self.psi[k],self.psi[k])
            self.psi[k] = self.psi[k]/integral**0.5


        
    def plotWFN(self, v):
        figure=plt.figure()
        plt.plot(self.x,self.psi[v],label=r'$\psi_v(x)$, k = ' + str(v))
        plt.title(r'$v=$'+ str(v) + r', $E_v$=' + '{:10.4f}'.format(self.E[v]))
        plt.legend()
        plt.xlabel(r'$x$(bohr)')
        plt.ylabel(r'$\psi(x)$')


In [155]:
m1 = 0.5
omega1 = 2
n=400
L1 = 20
solver = PDE_BV(m1,omega1, L1, npoints=n)
solver.calc()
solver.plotWFN(0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
solver.plotWFN(4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Richardson extrapolation

In [181]:
nlist=[100,200,300,400]
h_list = []
E0_list = []
print("{:4s} {:20s} {:20s}".format("n","h","E"))
for n in nlist:
    solver = PDE_BV(m1,omega1, L1, npoints=n)
    solver.calc()
    E0_list.append(solver.E[0])
    h_list.append(solver.h)
    print("{:4d} {:20.16f} {:20.16f}".format(n, solver.h, solver.E[0]))
    

n    h                    E                   
 100   0.2020202020202024   0.9974426831385571
 200   0.1005025125628141   0.9993683035175135
 300   0.0668896321070243   0.9997202828060194
 400   0.0501253132832087   0.9998429411393255
